In [1]:
import os
import hashlib
import base58
from ecdsa import SigningKey, SECP256k1
import bech32

In [2]:
# 1. Generate a 256-bit private key
private_key_bytes = os.urandom(32)

private_key_int = int.from_bytes(private_key_bytes) # Convert to decimal number
print("Private Key (decimal):", private_key_int)

private_key_hex = private_key_bytes.hex()
print("Private Key (hex):", private_key_hex)

wif = base58.b58encode(b'\x80' + private_key_bytes + b'\x01' + hashlib.sha256(hashlib.sha256(b'\x80' + private_key_bytes + b'\x01').digest()).digest()[:4]).decode()
print("Private Key (WIF):", wif) # used for wallet import format

Private Key (decimal): 39658300337339047826160927983396658939240316553684214058011780889441245260805
Private Key (hex): 57adcd735ee44075e85123f3ef8b698e922e1c79f64a23c7454bae9891d2ec05
Private Key (WIF): KzA9VZMLdNm6fzGTTsFeqKnok32ZWDuMahgr5Y5a83PG2JKGQenY


In [3]:
# 2. Generate public key using secp256k1
sk = SigningKey.from_string(private_key_bytes, curve=SECP256k1)
vk = sk.verifying_key

# Format uncompressed public key (0x04 + X + Y)
public_key_bytes = b'\x04' + vk.to_string()
public_key_int = int.from_bytes(public_key_bytes)
print("Public Key (decimal):", public_key_int)
public_key_hex = public_key_bytes.hex()
print("Public Key (hex):", public_key_hex)

Public Key (decimal): 58792602587016184377358809634919624120943859558511255682881716369226164813171417506092410885423566434700497989527501335725990365180956216770752950695791859
Public Key (hex): 04628c4159ce2a7da73c3e1246d7be945d681262be5b6da596fb5d528d3d31f98d6f4d4a5245dafd69833172da4c0d3439aa68931c609182604e9015cbb1ae8cf3


In [5]:
# 3. Create Bitcoin address (Legacy)
# Step 1: SHA-256 hash of the public key
sha256_hash = hashlib.sha256(public_key_bytes).digest()

# Step 2: RIPEMD-160 hash of the SHA-256 hash
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256_hash)
hashed_pubkey = ripemd160.digest()

# Step 3: Add version byte (0x00 for mainnet)
versioned_payload = b'\x00' + hashed_pubkey

# Step 4: Create checksum (double SHA-256)
checksum = hashlib.sha256(hashlib.sha256(versioned_payload).digest()).digest()[:4]

# Step 5: Combine and encode with Base58Check
full_payload = versioned_payload + checksum
address = base58.b58encode(full_payload).decode()

print("Bitcoin Address (Legacy):", address)

Bitcoin Address (Legacy): 1M9hEteoUDwQ5dCkVBXsdXtQAd8xrfJ9bY


In [6]:
# BITCOIN ADDRESS GENERATION

# 1. Compress the public key (required for SegWit)
pubkey_bytes = vk.to_string()
x = pubkey_bytes[:32]
y = pubkey_bytes[32:]
prefix = b'\x03' if int.from_bytes(y, 'big') % 2 else b'\x02'
compressed_pubkey = prefix + x
print("Compressed Public Key (hex):", compressed_pubkey.hex())

# 2. Hash of the compressed public key (SHA-256 then RIPEMD-160)
sha256 = hashlib.sha256(compressed_pubkey).digest()
ripemd160 = hashlib.new('ripemd160', sha256).digest()

# 3. Create Bitcoin address (P2WPKH - witness version 0)
witness_version = 0
witness_program = bech32.convertbits(ripemd160, 8, 5, True)
segwit_address = bech32.bech32_encode("bc", [witness_version] + witness_program)

print("Bitcoin Address (SegWit ver):", segwit_address)

Compressed Public Key (hex): 03628c4159ce2a7da73c3e1246d7be945d681262be5b6da596fb5d528d3d31f98d
Bitcoin Address (SegWit ver): bc1q9300wetpt2qs543sj9gvpuw74g0u358u346w6n
